In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
class AdvancedPortfolioManagement:
    def __init__(self, returns, cov_matrix, market_caps):
        """
        Initialize the class with returns, covariance matrix, and market capitalizations of assets.
        
        :param returns: Expected returns of assets
        :param cov_matrix: Covariance matrix of asset returns
        :param market_caps: Market capitalizations of assets
        """
        self.returns = returns
        self.cov_matrix = cov_matrix
        self.market_caps = market_caps
        self.market_weights = market_caps / np.sum(market_caps)
        self.tau = 0.05  # Scalar factor for prior uncertainty

    def factor_investing(self, factors):
        """
        Factor investing based on given factors.
        
        :param factors: DataFrame with factor loadings for each asset
        :return: Factor-based portfolio weights
        """
        factor_weights = factors.mean(axis=0)
        normalized_weights = factor_weights / np.sum(factor_weights)
        return normalized_weights
    
    def risk_parity(self):
        """
        Risk parity portfolio construction.
        
        :return: Risk parity portfolio weights
        """
        inv_vol = 1 / np.sqrt(np.diag(self.cov_matrix))
        risk_parity_weights = inv_vol / np.sum(inv_vol)
        return risk_parity_weights

    def black_litterman(self, P, Q, omega):
        """
        Black-Litterman model for portfolio optimization.
        
        :param P: Pick matrix for views (K x N)
        :param Q: View returns (K x 1)
        :param omega: Uncertainty in views (K x K diagonal matrix)
        :return: Black-Litterman adjusted expected returns and portfolio weights
        """
        # Step 1: Calculate implied equilibrium returns (pi)
        pi = self.tau * (self.cov_matrix @ self.market_weights)

        # Step 2: Calculate M, the combined uncertainty matrix
        tau_sigma_inv = np.linalg.inv(self.tau * self.cov_matrix)
        omega_inv = np.linalg.inv(omega)
        M_inverse = tau_sigma_inv + P.T @ omega_inv @ P
        M = np.linalg.inv(M_inverse)

        # Step 3: Calculate adjusted returns (mu)
        pi_term = tau_sigma_inv @ pi
        Q_term = P.T @ omega_inv @ Q
        adjusted_returns = M @ (pi_term + Q_term)

        # Step 4: Optimize portfolio with adjusted returns
        weights = self.mean_variance_optimization(adjusted_returns)

        return adjusted_returns, weights

    def mean_variance_optimization(self, expected_returns):
        """
        Mean-variance optimization to find the optimal portfolio weights.
        
        :param expected_returns: Adjusted expected returns from the Black-Litterman model
        :return: Optimal portfolio weights
        """
        cov_inv = np.linalg.inv(self.cov_matrix)
        weights = cov_inv @ expected_returns
        weights /= np.sum(weights)  # Normalize to ensure the weights sum to 1
        return weights

